<a href="https://colab.research.google.com/github/alexmancilla/Spam-email-classifier/blob/main/Spam_email_classifier%09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cargando los datos


In [ ]:
import os
import tarfile
import urllib.request

In [ ]:
import os

# Definir la URL base de descarga de los datos
DOWNLOAD_ROOT = "http://spamassassin.apache.org/old/publiccorpus/"

# Definir la URL completa para descargar el archivo de mensajes de correo no deseado legítimos (ham)
HAM_URL = DOWNLOAD_ROOT + "20030228_easy_ham.tar.bz2"

# Definir la URL completa para descargar el archivo de mensajes de correo no deseado (spam)
SPAM_URL = DOWNLOAD_ROOT + "20030228_spam.tar.bz2"

# Definir la ruta completa al directorio donde se almacenarán los datos descargados
SPAM_PATH = os.path.join("datasets", "spam")


In [ ]:
# Descargar y descomprimir archivos de spam en un directorio específico
def fetch_spam_data(ham_url=HAM_URL, spam_url=SPAM_URL, spam_path=SPAM_PATH):
    # Comprueba si el directorio spam_path no existe y, de ser así, lo crea
    if not os.path.isdir(spam_path):
        os.makedirs(spam_path)

    # Itera sobre los archivos y las URL correspondientes
    for filename, url in (("ham.tar.bz2", ham_url), ("spam.tar.bz2", spam_url)):
        # Obtiene la ruta completa del archivo en el directorio spam_path
        path = os.path.join(spam_path, filename)

        # Si el archivo no existe en la ruta, lo descarga desde la URL
        if not os.path.isfile(path):
            urllib.request.urlretrieve(url, path)

        # Abre el archivo tar.bz2 y extrae su contenido en el directorio spam_path
        tar_bz2_file = tarfile.open(path)
        tar_bz2_file.extractall(path=spam_path)
        tar_bz2_file.close()


In [ ]:
fetch_spam_data()

#Carga de los emails

In [ ]:
HAM_DIR = os.path.join(SPAM_PATH, "easy_ham")  # Ruta completa al directorio "easy_ham" dentro de SPAM_PATH
SPAM_DIR = os.path.join(SPAM_PATH, "spam")  # Ruta completa al directorio "spam" dentro de SPAM_PATH

# Obtener los nombres de archivo en HAM_DIR que tienen una longitud mayor a 20 caracteres
ham_filenames = [name for name in sorted(os.listdir(HAM_DIR)) if len(name) > 20]

# Obtener los nombres de archivo en SPAM_DIR que tienen una longitud mayor a 20 caracteres
spam_filenames = [name for name in sorted(os.listdir(SPAM_DIR)) if len(name) > 20]


In [ ]:
len(ham_filenames)

2500

In [ ]:
len(spam_filenames)

500

#Parsing email

Podemos utilizar el módulo `email` de Python para analizar estos correos electrónicos (se encarga de las cabeceras, la codificación, etc.):



In [ ]:
import email
import email.policy

In [ ]:
def load_email(is_spam, filename, spam_path=SPAM_PATH):
    # Determina el directorio en función de si el correo es spam o no
    directory = "spam" if is_spam else "easy_ham"

    # Abre el archivo de correo electrónico en modo de lectura binaria
    with open(os.path.join(spam_path, directory, filename), "rb") as f:
        # Utiliza BytesParser para analizar el contenido del archivo de correo electrónico
        return email.parser.BytesParser(policy=email.policy.default).parse(f)


In [ ]:
"""
Carga y distribución de los correos electrónicos no spam (ham)
y los correos electrónicos spam utilizando la función load_email().
Los resultados se almacenan en las listas ham_emails y spam_emails.
"""
ham_emails = [load_email(is_spam=False, filename=name) for name in ham_filenames]
spam_emails = [load_email(is_spam=True, filename=name) for name in spam_filenames]

##Ejemplo no spam

Veamos un ejemplo de no spam y otro de spam, para hacernos una idea de cómo son los datos:

In [ ]:
"""Imprime el contenido del segundo correo electrónico en la lista ham_emails
después de eliminar los espacios en blanco iniciales y finales."""
print(ham_emails[1].get_content().strip())

Martin A posted:
Tassos Papadopoulos, the Greek sculptor behind the plan, judged that the
 limestone of Mount Kerdylio, 70 miles east of Salonika and not far from the
 Mount Athos monastic community, was ideal for the patriotic sculpture. 
 
 As well as Alexander's granite features, 240 ft high and 170 ft wide, a
 museum, a restored amphitheatre and car park for admiring crowds are
planned
---------------------
So is this mountain limestone or granite?
If it's limestone, it'll weather pretty fast.

------------------------ Yahoo! Groups Sponsor ---------------------~-->
4 DVDs Free +s&p Join Now
http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM
---------------------------------------------------------------------~->

To unsubscribe from this group, send an email to:
forteana-unsubscribe@egroups.com

 

Your use of Yahoo! Groups is subject to http://docs.yahoo.com/info/terms/


##Ejemplo spam

In [ ]:
print(spam_emails[6].get_content().strip())

Help wanted.  We are a 14 year old fortune 500 company, that is
growing at a tremendous rate.  We are looking for individuals who
want to work from home.

This is an opportunity to make an excellent income.  No experience
is required.  We will train you.

So if you are looking to be employed from home with a career that has
vast opportunities, then go:

http://www.basetel.com/wealthnow

We are looking for energetic and self motivated people.  If that is you
than click on the link and fill out the form, and one of our
employement specialist will contact you.

To be removed from our link simple go to:

http://www.basetel.com/remove.html


4139vOLW7-758DoDY1425FRhM1-764SMFc8513fCsLl40


##Multiparts email

Algunos correos electrónicos son en realidad multiparte, con imágenes y archivos adjuntos (que pueden tener sus propios datos adjuntos). Veamos los distintos tipos de estructuras que tenemos:

In [ ]:
def get_email_structure(email):
    # Verifica si el argumento 'email' es una cadena de texto (correo sin estructura)
    if isinstance(email, str):
        return email

    # Obtiene el contenido del correo electrónico
    payload = email.get_payload()

    # Verifica si el contenido del correo es una lista (correo multipart)
    if isinstance(payload, list):
        # Construye una representación de cadena para correo multipart
        return "multipart({})".format(", ".join([
            get_email_structure(sub_email)
            for sub_email in payload
        ]))
    else:
        # Retorna el tipo de contenido del correo (por ejemplo, text/plain, text/html)
        return email.get_content_type()


In [ ]:
from collections import Counter

def structures_counter(emails):
    # Crea un objeto Counter para contar las estructuras de correo
    structures = Counter()

    # Itera sobre cada correo electrónico en la lista 'emails'
    for email in emails:
        # Obtiene la estructura del correo electrónico utilizando la función 'get_email_structure'
        structure = get_email_structure(email)

        # Incrementa el contador para la estructura correspondiente
        structures[structure] += 1

    # Retorna el objeto Counter con las estructuras de correo contadas
    return structures


##Estructura no spam

In [ ]:
structures_counter(ham_emails).most_common()

[('text/plain', 2408),
 ('multipart(text/plain, application/pgp-signature)', 66),
 ('multipart(text/plain, text/html)', 8),
 ('multipart(text/plain, text/plain)', 4),
 ('multipart(text/plain)', 3),
 ('multipart(text/plain, application/octet-stream)', 2),
 ('multipart(text/plain, text/enriched)', 1),
 ('multipart(text/plain, application/ms-tnef, text/plain)', 1),
 ('multipart(multipart(text/plain, text/plain, text/plain), application/pgp-signature)',
  1),
 ('multipart(text/plain, video/mng)', 1),
 ('multipart(text/plain, multipart(text/plain))', 1),
 ('multipart(text/plain, application/x-pkcs7-signature)', 1),
 ('multipart(text/plain, multipart(text/plain, text/plain), text/rfc822-headers)',
  1),
 ('multipart(text/plain, multipart(text/plain, text/plain), multipart(multipart(text/plain, application/x-pkcs7-signature)))',
  1),
 ('multipart(text/plain, application/x-java-applet)', 1)]

##Estructura SPAM

In [ ]:
structures_counter(spam_emails).most_common()

[('text/plain', 218),
 ('text/html', 183),
 ('multipart(text/plain, text/html)', 45),
 ('multipart(text/html)', 20),
 ('multipart(text/plain)', 19),
 ('multipart(multipart(text/html))', 5),
 ('multipart(text/plain, image/jpeg)', 3),
 ('multipart(text/html, application/octet-stream)', 2),
 ('multipart(text/plain, application/octet-stream)', 1),
 ('multipart(text/html, text/plain)', 1),
 ('multipart(multipart(text/html), application/octet-stream, image/jpeg)', 1),
 ('multipart(multipart(text/plain, text/html), image/gif)', 1),
 ('multipart/alternative', 1)]

Parece que los correos electrónicos `no spam` son más a menudo texto plano, mientras que el spam tiene bastante HTML. Además, bastantes correos electrónicos de spam están firmados con PGP, mientras que ninguno lo está. En resumen, parece que la estructura del correo electrónico es una información útil.

###Headers

Ahora echemos un vistazo a las cabeceras del correo electrónico:

In [ ]:
"""el código recorre los elementos del diccionario de encabezados del primer correo electrónico
en la lista spam_emails
e imprime cada encabezado junto con su valor correspondiente."""
for header, value in spam_emails[0].items():
    print(header,":",value)

Return-Path : <12a1mailbot1@web.de>
Delivered-To : zzzz@localhost.spamassassin.taint.org
Received : from localhost (localhost [127.0.0.1])	by phobos.labs.spamassassin.taint.org (Postfix) with ESMTP id 136B943C32	for <zzzz@localhost>; Thu, 22 Aug 2002 08:17:21 -0400 (EDT)
Received : from mail.webnote.net [193.120.211.219]	by localhost with POP3 (fetchmail-5.9.0)	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 13:17:21 +0100 (IST)
Received : from dd_it7 ([210.97.77.167])	by webnote.net (8.9.3/8.9.3) with ESMTP id NAA04623	for <zzzz@spamassassin.taint.org>; Thu, 22 Aug 2002 13:09:41 +0100
From : 12a1mailbot1@web.de
Received : from r-smtp.korea.com - 203.122.2.197 by dd_it7  with Microsoft SMTPSVC(5.5.1775.675.6);	 Sat, 24 Aug 2002 09:42:10 +0900
To : dcek1a1@netsgo.com
Subject : Life Insurance - Why Pay More?
Date : Wed, 21 Aug 2002 20:31:57 -1600
MIME-Version : 1.0
Message-ID : <0103c1042001882DD_IT7@dd_it7>
Content-Type : text/html; charset="iso-8859-1"
Content-Transfer-Encoding : qu

Probablemente haya mucha información útil, como la dirección de correo electrónico del remitente (12a1mailbot1@web.de parece sospechosa), pero nos centraremos en la cabecera `Subject`:

In [ ]:
"""el código recorre los elementos del diccionario de encabezados del primer correo electrónico
en la lista ham_emails
e imprime cada encabezado junto con su valor correspondiente."""
for header, value in ham_emails[0].items():
    print(header,":",value)

Return-Path : <exmh-workers-admin@spamassassin.taint.org>
Delivered-To : zzzz@localhost.netnoteinc.com
Received : from localhost (localhost [127.0.0.1])	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id D03E543C36	for <zzzz@localhost>; Thu, 22 Aug 2002 07:36:16 -0400 (EDT)
Received : from phobos [127.0.0.1]	by localhost with IMAP (fetchmail-5.9.0)	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 12:36:16 +0100 (IST)
Received : from listman.spamassassin.taint.org (listman.spamassassin.taint.org [66.187.233.211]) by    dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g7MBYrZ04811 for    <zzzz-exmh@spamassassin.taint.org>; Thu, 22 Aug 2002 12:34:53 +0100
Received : from listman.spamassassin.taint.org (localhost.localdomain [127.0.0.1]) by    listman.redhat.com (Postfix) with ESMTP id 8386540858; Thu, 22 Aug 2002    07:35:02 -0400 (EDT)
Delivered-To : exmh-workers@listman.spamassassin.taint.org
Received : from int-mx1.corp.spamassassin.taint.org (int-mx1.corp.spamassassin.taint.org 

In [ ]:
spam_emails[0]["Subject"]

'Life Insurance - Why Pay More?'

#Empezando

##Spliting dataset

split it into a training set and a test set:

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
X = np.array(ham_emails + spam_emails, dtype=object)
y = np.array([0] * len(ham_emails) + [1] * len(spam_emails))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

*funciones* de preprocesamiento.
Primero, necesitaremos una función para convertir HTML a texto sin formato. Podría decirse que la mejor manera de hacer esto sería usar la gran biblioteca [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/), pero me gustaría evitar agregar otra dependencia a este proyecto, así que vamos a hackear una solución rápida y sucia usando expresiones regulares (a riesgo de [un̨ho͞ly radiańcé destro҉ying all enli̍̈́̂̈́ghtenment](https://stackoverflow.com/a/1732454/38626)). La siguiente función primero elimina la sección `<head>`, luego convierte todas las etiquetas `<a>` a la palabra HIPERVÍNCULO, luego elimina todas las etiquetas HTML, dejando solo el texto sin formato. Para mejorar la legibilidad, también reemplaza varias líneas nuevas con líneas nuevas únicas y, finalmente, elimina las entidades html (como `&gt;` o `&nbsp;`):

##Pre-procesado

In [ ]:
import re
from html import unescape

def html_to_plain_text(html):
    # Elimina la sección <head> y su contenido
    text = re.sub('<head.*?>.*?</head>', '', html, flags=re.M | re.S | re.I)

    # Reemplaza las etiquetas <a> con el texto "HYPERLINK"
    text = re.sub('<a\s.*?>', ' HYPERLINK ', text, flags=re.M | re.S | re.I)

    # Elimina todas las demás etiquetas HTML
    text = re.sub('<.*?>', '', text, flags=re.M | re.S)

    # Reemplaza múltiples saltos de línea y espacios en blanco con un solo salto de línea
    text = re.sub(r'(\s*\n)+', '\n', text, flags=re.M | re.S)

    # Decodifica los caracteres HTML especiales en su forma original
    return unescape(text)


Prueba con HTML spam:

In [ ]:
html_spam_emails = [email for email in X_train[y_train==1]
                    if get_email_structure(email) == "text/html"]
sample_html_spam = html_spam_emails[7]
print(sample_html_spam.get_content().strip()[:1000], "...")

<HTML><HEAD><TITLE></TITLE><META http-equiv="Content-Type" content="text/html; charset=windows-1252"><STYLE>A:link {TEX-DECORATION: none}A:active {TEXT-DECORATION: none}A:visited {TEXT-DECORATION: none}A:hover {COLOR: #0033ff; TEXT-DECORATION: underline}</STYLE><META content="MSHTML 6.00.2713.1100" name="GENERATOR"></HEAD>
<BODY text="#000000" vLink="#0033ff" link="#0033ff" bgColor="#CCCC99"><TABLE borderColor="#660000" cellSpacing="0" cellPadding="0" border="0" width="100%"><TR><TD bgColor="#CCCC99" valign="top" colspan="2" height="27">
<font size="6" face="Arial, Helvetica, sans-serif" color="#660000">
<b>OTC</b></font></TD></TR><TR><TD height="2" bgcolor="#6a694f">
<font size="5" face="Times New Roman, Times, serif" color="#FFFFFF">
<b>&nbsp;Newsletter</b></font></TD><TD height="2" bgcolor="#6a694f"><div align="right"><font color="#FFFFFF">
<b>Discover Tomorrow's Winners&nbsp;</b></font></div></TD></TR><TR><TD height="25" colspan="2" bgcolor="#CCCC99"><table width="100%" border="0" 

Resultados después de parsear a plain text:

In [ ]:
print(html_to_plain_text(sample_html_spam.get_content())[:1000], "...")


OTC
 Newsletter
Discover Tomorrow's Winners 
For Immediate Release
Cal-Bay (Stock Symbol: CBYI)
Watch for analyst "Strong Buy Recommendations" and several advisory newsletters picking CBYI.  CBYI has filed to be traded on the OTCBB, share prices historically INCREASE when companies get listed on this larger trading exchange. CBYI is trading around 25 cents and should skyrocket to $2.66 - $3.25 a share in the near future.
Put CBYI on your watch list, acquire a position TODAY.
REASONS TO INVEST IN CBYI
A profitable company and is on track to beat ALL earnings estimates!
One of the FASTEST growing distributors in environmental & safety equipment instruments.
Excellent management team, several EXCLUSIVE contracts.  IMPRESSIVE client list including the U.S. Air Force, Anheuser-Busch, Chevron Refining and Mitsubishi Heavy Industries, GE-Energy & Environmental Research.
RAPIDLY GROWING INDUSTRY
Industry revenues exceed $900 million, estimates indicate that there could be as much as $25 billi

Genial. Ahora escribamos una función que tome un correo electrónico como entrada y devuelva su contenido como texto plano, sea cual sea su formato:

In [ ]:
def email_to_text(email):
    html = None

    # Itera sobre cada parte del correo electrónico
    for part in email.walk():
        ctype = part.get_content_type()

        # Verifica si el tipo de contenido es "text/plain" o "text/html"
        if not ctype in ("text/plain", "text/html"):
            continue

        try:
            # Obtiene el contenido de la parte
            content = part.get_content()
        except:  # en caso de problemas de codificación
            content = str(part.get_payload())

        # Si el tipo de contenido es "text/plain", retorna el contenido
        if ctype == "text/plain":
            return content
        else:
            # Si el tipo de contenido es "text/html", almacena el contenido HTML
            html = content

    # Si se encontró contenido HTML, convierte el HTML a texto plano
    if html:
        return html_to_plain_text(html)


In [ ]:
print(email_to_text(sample_html_spam)[:100], "...")


OTC
 Newsletter
Discover Tomorrow's Winners 
For Immediate Release
Cal-Bay (Stock Symbol: CBYI)
Wat ...


### Steaming

In [ ]:
try:
    import nltk

    # Importa el módulo NLTK y crea una instancia de PorterStemmer
    stemmer = nltk.PorterStemmer()

    # Itera sobre una lista de palabras para realizar el stemming
    for word in ("Computations", "Computation", "Computing", "Computed", "Compute", "Compulsive"):
        # Aplica el stemming a cada palabra y muestra el resultado
        print(word, "=>", stemmer.stem(word))
except ImportError:
    # Si ocurre un error de importación, muestra un mensaje de error
    print("Error: stemming requires the NLTK module.")
    stemmer = None


Computations => comput
Computation => comput
Computing => comput
Computed => comput
Compute => comput
Compulsive => compuls


También necesitaremos una forma de reemplazar URLs con la palabra "URL". Para esto, podríamos usar [regular expressions](https://mathiasbynens.be/demo/url-regex) pero usaremos la librería [urlextract](https://github.com/lipoja/URLExtract). Puedes instalarla con el siguiente comando (no olvides activar tu virtualenv primero; si no tienes uno, probablemente necesitarás derechos de administrador, o usar la opción `--user`):

In [ ]:
%pip install -q -U urlextract

In [ ]:
try:
    import urlextract

    # Importa el módulo urlextract y crea una instancia de URLExtract
    url_extractor = urlextract.URLExtract()

    # Aplica el extractor de URL a un texto de ejemplo
    extracted_urls = url_extractor.find_urls("Will it detect github.com and https://youtu.be/7Pq-S557XQU?t=3m32s")

    # Imprime las URL extraídas
    print(extracted_urls)
except ImportError:
    # Si ocurre un error de importación, muestra un mensaje de error
    print("Error: replacing URLs requires the urlextract module.")
    url_extractor = None


['github.com', 'https://youtu.be/7Pq-S557XQU?t=3m32s']


Estamos listos para juntar todo esto en un transformer que usaremos para convertir correos electrónicos en contadores de palabras. Tenga en cuenta que dividimos las frases en palabras utilizando el método `split()` de Python, que utiliza espacios en blanco para los límites de las palabras. Esto funciona para muchos idiomas escritos, pero no para todos. Por ejemplo, las escrituras chinas y japonesas generalmente no usan espacios entre palabras, y el vietnamita a menudo usa espacios incluso entre sílabas. En este ejercicio no hay problema, porque el conjunto de datos está (en su mayoría) en inglés.

###Transformer

EmailToWordCounterTransformer, hereda de las clases `BaseEstimator` y `TransformerMixin` del paquete sklearn.base. Esta clase es un transformador personalizado que se puede usar en un pipeline de Scikit-Learn para preprocesar correos electrónicos.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class EmailToWordCounterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, strip_headers=True, lower_case=True, remove_punctuation=True,
                 replace_urls=True, replace_numbers=True, stemming=True):
        # Inicializa los parámetros del transformador
        self.strip_headers = strip_headers
        self.lower_case = lower_case
        self.remove_punctuation = remove_punctuation
        self.replace_urls = replace_urls
        self.replace_numbers = replace_numbers
        self.stemming = stemming

    def fit(self, X, y=None):
        # No se requiere ajuste, simplemente retorna el objeto transformador
        return self

    def transform(self, X, y=None):
        # Transforma los datos de entrada X
        X_transformed = []

        # Itera sobre cada correo electrónico en X
        for email in X:
            # Convierte el correo electrónico en texto plano utilizando la función email_to_text
            text = email_to_text(email) or ""

            # Aplica las transformaciones especificadas en los parámetros del transformador
            if self.lower_case:
                text = text.lower()

            if self.replace_urls and url_extractor is not None:
                # Reemplaza las URLs por la palabra "URL" si la opción replace_urls está habilitada y el extractor de URL está disponible
                urls = list(set(url_extractor.find_urls(text)))
                urls.sort(key=lambda url: len(url), reverse=True)
                for url in urls:
                    text = text.replace(url, " URL ")

            if self.replace_numbers:
                # Reemplaza los números por la palabra "NUMBER" si la opción replace_numbers está habilitada
                text = re.sub(r'\d+(?:\.\d*)?(?:[eE][+-]?\d+)?', 'NUMBER', text)

            if self.remove_punctuation:
                # Elimina la puntuación si la opción remove_punctuation está habilitada
                text = re.sub(r'\W+', ' ', text, flags=re.M)

            # Cuenta la frecuencia de las palabras en el texto
            word_counts = Counter(text.split())

            if self.stemming and stemmer is not None:
                # Realiza stemming en las palabras si la opción stemming está habilitada y el objeto stemmer está disponible
                stemmed_word_counts = Counter()
                for word, count in word_counts.items():
                    stemmed_word = stemmer.stem(word)
                    stemmed_word_counts[stemmed_word] += count
                word_counts = stemmed_word_counts

            # Agrega las cuentas de palabras transformadas a la lista de resultados
            X_transformed.append(word_counts)

        # Retorna los datos transformados en forma de matriz numpy
        return np.array(X_transformed)


### Prueba

*Probemos este transformador en algunos correos:*

In [ ]:
X_few = X_train[:3]
X_few_wordcounts = EmailToWordCounterTransformer().fit_transform(X_few)
X_few_wordcounts

array([Counter({'chuck': 1, 'murcko': 1, 'wrote': 1, 'stuff': 1, 'yawn': 1, 'r': 1}),
       Counter({'the': 11, 'of': 9, 'and': 8, 'all': 3, 'christian': 3, 'to': 3, 'by': 3, 'jefferson': 2, 'i': 2, 'have': 2, 'superstit': 2, 'one': 2, 'on': 2, 'been': 2, 'ha': 2, 'half': 2, 'rogueri': 2, 'teach': 2, 'jesu': 2, 'some': 1, 'interest': 1, 'quot': 1, 'url': 1, 'thoma': 1, 'examin': 1, 'known': 1, 'word': 1, 'do': 1, 'not': 1, 'find': 1, 'in': 1, 'our': 1, 'particular': 1, 'redeem': 1, 'featur': 1, 'they': 1, 'are': 1, 'alik': 1, 'found': 1, 'fabl': 1, 'mytholog': 1, 'million': 1, 'innoc': 1, 'men': 1, 'women': 1, 'children': 1, 'sinc': 1, 'introduct': 1, 'burnt': 1, 'tortur': 1, 'fine': 1, 'imprison': 1, 'what': 1, 'effect': 1, 'thi': 1, 'coercion': 1, 'make': 1, 'world': 1, 'fool': 1, 'other': 1, 'hypocrit': 1, 'support': 1, 'error': 1, 'over': 1, 'earth': 1, 'six': 1, 'histor': 1, 'american': 1, 'john': 1, 'e': 1, 'remsburg': 1, 'letter': 1, 'william': 1, 'short': 1, 'again': 1, 'becom

Ahora tenemos el recuento de palabras y necesitamos convertirlo en vectores. Para ello, construiremos otro transformador cuyo método `fit()` construirá el vocabulario (una lista ordenada de las palabras más comunes) y cuyo método `transform()` utilizará el vocabulario para convertir los recuentos de palabras en vectores. El resultado es una matriz dispersa.

In [ ]:
from scipy.sparse import csr_matrix

class WordCounterToVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size=1000):
        # Inicializa el tamaño del vocabulario
        self.vocabulary_size = vocabulary_size

    def fit(self, X, y=None):
        # Cuenta la frecuencia total de las palabras en los datos de entrada X
        total_count = Counter()
        for word_count in X:
            for word, count in word_count.items():
                total_count[word] += min(count, 10)

        # Obtiene las palabras más comunes hasta el tamaño del vocabulario
        most_common = total_count.most_common()[:self.vocabulary_size]

        # Crea un diccionario de vocabulario asignando un índice a cada palabra
        self.vocabulary_ = {word: index + 1 for index, (word, count) in enumerate(most_common)}

        # Retorna el objeto transformador
        return self

    def transform(self, X, y=None):
        # Construye una matriz dispersa comprimida (CSR matrix) a partir de los datos transformados
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)

        # Crea una matriz dispersa comprimida (CSR matrix) con los datos y la forma adecuada
        return csr_matrix((data, (rows, cols)), shape=(len(X), self.vocabulary_size + 1))


In [ ]:
"""
utiliza una instancia del transformador WordCounterToVectorTransformer para
 transformar los recuentos de palabras X_few_wordcounts en una representación
 vectorial utilizando un tamaño de vocabulario de 10."""

vocab_transformer = WordCounterToVectorTransformer(vocabulary_size=10)
X_few_vectors = vocab_transformer.fit_transform(X_few_wordcounts)
X_few_vectors

<3x11 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [ ]:
"""
´X_few_vectors.toarray():´ Llama al método toarray() en la matriz dispersa X_few_vectors para convertirla en una matriz densa,
que es una representación estándar de una matriz en la que todos los elementos se almacenan en memoria.
El resultado es una matriz densa que contiene la representación vectorial de los datos X_few_wordcounts.
"""
X_few_vectors.toarray()

array([[ 6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [99, 11,  9,  8,  3,  1,  3,  1,  3,  2,  3],
       [67,  0,  1,  2,  3,  4,  1,  2,  0,  1,  0]])

¿Qué significa esta matriz?

1.   Bien, el 99 de la segunda fila, primera columna,forman parte del vocabulario.
2.     El 11 al lado significa que la primera palabra del vocabulario está presente 11 veces en este correo electrónico.
3.   El 9 al lado significa que la segunda palabra está presente 9 veces, y así sucesivamente.
Puedes consultar el vocabulario para saber de qué palabras estamos hablando. La primera palabra es "the", la segunda palabra es "of", etc.

In [ ]:
"""
Devuelve el diccionario de vocabulario creado por el transformador WordCounterToVectorTransformer durante el ajuste.
Proporciona una asignación de palabras a índices en la representación vectorial.
"""
vocab_transformer.vocabulary_

{'the': 1,
 'of': 2,
 'and': 3,
 'to': 4,
 'url': 5,
 'all': 6,
 'in': 7,
 'christian': 8,
 'on': 9,
 'by': 10}

##Entrenamieto

Ya estamos listos para entrenar nuestro primer clasificador de spam. Transformemos todo el conjunto de datos:


El código que proporcionaste utiliza la clase `Pipeline` de scikit-learn para crear un pipeline de procesamiento de datos que incluye dos transformadores: `EmailToWordCounterTransformer` y `WordCounterToVectorTransformer`. Luego, se aplica el pipeline al conjunto de entrenamiento `X_train` para obtener los datos transformados `X_train_transformed`.

In [ ]:
from sklearn.pipeline import Pipeline

# Crea un pipeline de procesamiento de datos con los transformadores EmailToWordCounterTransformer y WordCounterToVectorTransformer
preprocess_pipeline = Pipeline([
    ("email_to_wordcount", EmailToWordCounterTransformer()),  # Transformador 1: EmailToWordCounterTransformer
    ("wordcount_to_vector", WordCounterToVectorTransformer()),  # Transformador 2: WordCounterToVectorTransformer
])

# Aplica el pipeline al conjunto de entrenamiento X_train
X_train_transformed = preprocess_pipeline.fit_transform(X_train)


Nota: para estar preparados para el futuro, establecemos `solver="lbfgs"` ya que este será el valor por defecto en Scikit-Learn 0.22.
--- ---



##Resultados

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Crea una instancia del clasificador LogisticRegression
log_clf = LogisticRegression(solver="lbfgs", max_iter=1000, random_state=42)

# Realiza validación cruzada en los datos transformados y las etiquetas
score = cross_val_score(log_clf, X_train_transformed, y_train, cv=3, verbose=3)

# Calcula el promedio de las puntuaciones obtenidas en la validación cruzada
score.mean()


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] END ................................ score: (test=0.981) total time=   0.2s
[CV] END ................................ score: (test=0.984) total time=   0.2s


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s


[CV] END ................................ score: (test=0.990) total time=   0.3s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.6s finished


0.985

In [ ]:
score.mean()

0.985

In [ ]:
from sklearn.metrics import precision_score, recall_score

# Transforma los datos de prueba utilizando el pipeline de preprocesamiento
X_test_transformed = preprocess_pipeline.transform(X_test)

# Crea una instancia del clasificador LogisticRegression
log_clf = LogisticRegression(solver="lbfgs", max_iter=1000, random_state=42)

# Ajusta el clasificador a los datos de entrenamiento transformados
log_clf.fit(X_train_transformed, y_train)

# Realiza predicciones en los datos de prueba transformados
y_pred = log_clf.predict(X_test_transformed)

# Calcula y muestra la precisión y la recuperación
print("Precision: {:.2f}%".format(100 * precision_score(y_test, y_pred)))
print("Recall: {:.2f}%".format(100 * recall_score(y_test, y_pred)))


Precision: 96.88%
Recall: 97.89%
